In [1]:
d= {"key": "val", "k1": "v1"}

In [2]:
d["key"]

'val'

In [3]:
from box import ConfigBox

In [4]:
d2 = ConfigBox({"key": "val", "k1": "v1"})

In [5]:
d2

ConfigBox({'key': 'val', 'k1': 'v1'})

In [6]:
d2.key

'val'

In [7]:
def get_products(x:int, y:int) -> int:
    return x*y

In [8]:
get_products(2,3)

6

In [9]:
get_products(2,"3")

'33'

In [10]:
from ensure import ensure_annotations

In [11]:
@ensure_annotations
def get_products(x:int, y:int) -> int:
    return x*y

In [12]:
get_products(2,"3")

EnsureError: Argument y of type <class 'str'> to <function get_products at 0x000001E62AF857E0> does not match annotation type <class 'int'>

### NLP TRial

In [2]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
from nltk.tokenize import sent_tokenize
from tqdm import tqdm
import torch

nltk.download('punkt')

c:\Users\Suraj\Desktop\Python\Text-Summarizer-Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Suraj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import os
import gc

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# Clear cache
gc.collect()
torch.cuda.empty_cache()

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = torch.device("cuda:0")

model_ckpt = "google/pegasus-cnn_dailymail"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt)
# model = torch.nn.DataParallel(model)
# model.to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\Suraj\Desktop\Python\Text-Summarizer-Project\venv\lib\site-packages\torch\nn\modules\module.py:1326: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  return t.to(


In [3]:
torch.cuda.is_available()

True

In [16]:
# download and unzip dataset

import requests
import zipfile
import os

# Download the dataset
url = 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip'
response = requests.get(url)

# Save the zip file
zip_file_path = 'summarizer-data.zip'
with open(zip_file_path, 'wb') as f:
    f.write(response.content)

# Unzip the file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall('summarizer-data')

# Optionally, remove the zip file after extraction
os.remove(zip_file_path)


In [4]:
data_samsum = load_from_disk('../data/summarizer-data/samsum_dataset')

In [5]:
data_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [6]:
split_lengths = [len(data_samsum[split]) for split in data_samsum]

print(f"Split lengths: {split_lengths}")
print(f"Features: {data_samsum['train'].column_names}")
print("\nDialogue:")
print(data_samsum["train"][0]["dialogue"])
print("\nSummary:")
print(data_samsum["train"][0]["summary"])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-)

Summary:
Amanda baked cookies and will bring Jerry some tomorrow.


In [16]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True, padding='max_length')

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch["summary"], max_length=128, truncation=True, padding='max_length')

    return {
        "input_ids": input_encodings["input_ids"],
        "attention_mask": input_encodings["attention_mask"],
        "labels": target_encodings["input_ids"],
    }

In [17]:
dataset_samsum_pt = data_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map: 100%|██████████| 14732/14732 [00:09<00:00, 1611.99 examples/s]


In [19]:
# Training

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [20]:
from transformers import TrainingArguments, Trainer

! pip install accelerate

training_args = TrainingArguments(
    output_dir="pegasus-samsum", num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1, weight_decay=0.01,
    logging_steps=10, evaluation_strategy="steps", eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=8, fp16=True, max_steps=1000, remove_unused_columns=False
)

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
    train_dataset=dataset_samsum_pt["train"],
    eval_dataset=dataset_samsum_pt["validation"],
)

C:\Users\Suraj\AppData\Local\Temp\ipykernel_4736\227094938.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
max_steps is given, it will override any value given in num_train_epochs


In [23]:
import gc

gc.collect()

torch.cuda.empty_cache()

trainer.train()

  0%|          | 0/1000 [00:23<?, ?it/s]


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`id` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [ ]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score